In this notebook, we'll look at building a visualiser to view the VPU applied en-mass to FFT data.

We can make our SensorSource objects iterables that return a next frame of data - https://www.programiz.com/python-programming/iterator ```__iter__``` just returns self (with any initialisation) and ```__next__``` returns self.read().

Our SensorSource objects also need a way of returning the size of the frame.

# Testing Sensor Sources

In [2]:
from src.sources.capture import VideoSource

video = VideoSource()
video.start()
grabbed, frame = video.read()
print(grabbed)
print(frame.shape)

video.stop()

True
(480, 640, 3)


In [4]:
from src.sources.capture import AudioSource
import time
import numpy as np

audio = AudioSource()
audio.start()
time.sleep(0.5)
# Test read
length1, samples1 = audio.read()
assert length1
assert samples1.any()
# Check starting and getting a frame
audio.start()
time.sleep(0.5)
length2, samples2 = audio.read()
assert length2
assert not np.array_equal(samples1, samples2)
print(samples2.shape)
# Test stopping
audio.stop()
assert not audio.started

[!] Asynchroneous capturing has already been started.
(65536,)


In [5]:
from src.sources.capture import CombinedSource, SensorSource

combined = CombinedSource()
type(combined.sources) == dict

True

In [6]:
len(combined.sources)

0

In [7]:
assert type(combined.sources) == dict
assert len(combined.sources) == 0
# Adding a source
combined.add_source(SensorSource())

In [8]:
list(combined.sources.items())[0][0]

'SensorSource'

In [9]:
from src.sources.capture import AVCapture

av = AVCapture()
av.start()
time.sleep(0.25)
data = av.read()
print(data)
av.stop()

{'audio': array([     0,      0,      0, ...,  -8800,  -8815, -10055], dtype=int16), 'video': array([[[181, 174, 127],
        [181, 174, 127],
        [182, 174, 130],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[181, 174, 127],
        [181, 174, 127],
        [184, 174, 130],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[183, 173, 127],
        [183, 173, 127],
        [184, 174, 130],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[129, 108,  82],
        [129, 108,  82],
        [128, 107,  88],
        ...,
        [ 58,  52,  62],
        [ 58,  55,  56],
        [ 59,  57,  57]],

       [[128, 107,  81],
        [129, 108,  82],
        [128, 107,  88],
        ...,
        [ 54,  51,  58],
        [ 58,  53,  54],
        [ 60,  55,  56]],

       [[128, 108,  79],
        [128, 108,  79],
        [128, 110,

# Test Covariance Unit

In [49]:
from src.var_processor.covariance import CovarianceUnit
cov_unit = CovarianceUnit(2)
print(cov_unit.x_sum, cov_unit.square_sum, cov_unit.covariance, sep="\n", end="\n---\n")
assert not cov_unit.x_sum.any()
assert not cov_unit.square_sum.any()
# Test updating with data
ones = np.ones(shape=(2,1))
cov_unit.update(ones)
assert cov_unit.count == 1
assert np.array_equal(cov_unit.x_sum, ones)
assert np.array_equal(cov_unit.mean, ones)
assert not cov_unit.covariance.any()
print(cov_unit.x_sum, cov_unit.square_sum, cov_unit.covariance, sep="\n", end="\n---\n")
threes = ones*3
cov_unit.update(threes)
assert cov_unit.count == 2
assert np.array_equal(cov_unit.x_sum, ones+threes)
assert cov_unit.square_sum.any()
assert np.array_equal(cov_unit.mean, ones*2)
assert cov_unit.covariance.any()
print(cov_unit.x_sum, cov_unit.square_sum, cov_unit.covariance, sep="\n", end="\n---\n")

[[0.]
 [0.]]
[[0. 0.]
 [0. 0.]]
[[nan nan]
 [nan nan]]
---
[[1.]
 [1.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
---
[[4.]
 [4.]]
[[0.66666667 0.66666667]
 [0.66666667 0.66666667]]
[[0.33333333 0.33333333]
 [0.33333333 0.33333333]]
---


/home/ben/projects/var_processor/src/var_processor/covariance.py:40: RuntimeWarning: invalid value encountered in true_divide
  return self.square_sum / self.count


In [10]:
from src.var_processor.power_iterator import PowerIterator

power = PowerIterator(2)
power.ev

array([[0.89852016],
       [0.43893226]])

In [11]:
from src.var_processor.vpu import VPU

# Intialise VPU
vpu = VPU(2)
# Test Iteration
in_1 = np.random.randint(255, size=(2, 1))
in_1 = in_1 / in_1.max()
print(in_1)
r, residual = vpu.iterate(in_1)
print(r, residual)
r, residual = vpu.iterate(in_1)
print(r, residual)

[[0.24691358]
 [1.        ]]
[[1.01018031]] [[ 0.20104366]
 [-0.00913835]]
[[1.01018031]] [[ 0.20104366]
 [-0.00913835]]


In [12]:
for _ in range(0, 100):
    in_1 = np.random.randint(255, size=(2, 1))
    in_1 = in_1 / in_1.max()
    vpu.update_cov(in_1)
r, residual = vpu.iterate(in_1)
print(r, residual)

[[0.37425228]] [[1.18382669]
 [0.66755374]]


In [13]:
vpu.cu.covariance

array([[ 0.10436427, -0.05691703],
       [-0.05691703,  0.09511006]])

In [14]:
vpu.pi.eigenvector

array([[-0.4911839],
       [ 0.8710559]])

In [15]:
vpu.pi.cov

array([[ 0.10436427, -0.05691703],
       [-0.05691703,  0.09511006]])

In [16]:
self = vpu.pi
self.ev = np.matmul(np.power(self.cov, 1), self.ev)
# Scale to have unit length (convert to integer values?)
# self.ev = self.ev / np.linalg.norm(self.ev)
print(self.ev)

[[-0.10083996]
 [ 0.11080291]]


In [17]:
np.power(self.cov, 1)

array([[ 0.10436427, -0.05691703],
       [-0.05691703,  0.09511006]])

In [18]:
self.ev

array([[-0.10083996],
       [ 0.11080291]])

Ah this is it - if self.ev becomes nan it stays as nan. Need a check to prevent this

In [104]:
# Intialise VPU
vpu2 = VPU(2)
# Test Iteration
for _ in range(0, 100):
    data_in = np.random.randint(2, size=(2, 1))
    cause, residual = vpu2.iterate(data_in)
print(cause, residual)
print(vpu2.cu.covariance)
assert vpu2.cu.covariance.any()
assert cause.any()
assert residual.any()
vpu2.reset()
# assert not vpu2.cu.covariance.any()
print(vpu2.cu.covariance)


[[-0.71491043]] [[0.49987684]
 [0.48890307]]
[[ 0.22997666 -0.01745778]
 [-0.01745778  0.231134  ]]
[[nan nan]
 [nan nan]]


# Sensor Object

The input into each VPU is a vector of N. Whatever the stage.

Class object. 
* Internal variables (for init)
    * vec_len - vector length (N)
    * time_len - length of time buffering (M)
* Methods
    * generate_stage - create a new time stage.
    * get_frame - get a frame of data from the sensor. 
        * Return:
            * frame
    


Stage.
* Internal variables (for init)
    * vec_len - vector length (N)
* Methods
    * __init__ - initialise a set of VPUs for one time stage. 
        * Input
            * stage_len - number of stages (k)
            * vec_len - vector length (N)
    * forward - input data and update VPUs.
        * Input:
            * stage_in - array of input data for stage.
        * Return:  
            * updated Rs and residuals for the stage as numpy array
    * get_cause - get the Rs from all individual VPUs. In binary form or float form?
        * Return:
            * causes - numpy array of Rs

We might actually want a "stage" object. generate_stage and process_stage are "stage" methods.

Time stages = logN(sensor_resolution)

We need a common way of getting the sensor_resolution. First time stage has sensor_resolution/N VPUs.

See:
* https://github.com/benhoyle/predictive_coding/blob/master/2019-10-28%20SpaceTime%20Grid%20object%20development.ipynb
* https://github.com/benhoyle/predictive_coding/blob/master/Time%20Filtering.ipynb

Below is very similar to our "layer" in the predictive_coding brain code. But we are flattening everything to 1D.

In [19]:
from src.var_processor.vpu import VPU

class TimeStage:
    """Object to represent a time stage of processing."""
    
    def __init__(self, vec_len, stage_len):
        """Initialise stage.
        
        Arg:
            vec_len - length of each 1D vector processed by the VPUs.
            stage_len - integer indicating number of VPUs.
        """
        self.vec_len = vec_len
        self.stage_len = stage_len
        self.size = self.vec_len*self.stage_len
        self.vpus = [VPU(vec_len) for _ in range(0, stage_len)]
        # Create a blank array for the causes
        self.causes = np.zeros(shape=(stage_len, 1))
        # Create a blank array for the residuals
        self.residuals = np.zeros(shape=(self.size, 1))
        
    def forward(self, stage_in):
        """Pass data to the stage for processing.
        
        Arg:
            stage_in - 1D numpy array with data to process.
        """
        # Create blank array to hold / pad data
        
        input_array = np.zeros(shape=(self.size, 1))
        # Check data is of right size
        if stage_in.shape[0] > self.size:
            # Crop input
            input_array = stage_in[:self.size]
        elif stage_in.shape[0] < self.size:
            input_array[:self.size] = stage_in
        else:
            input_array = stage_in
        # Iterate through VPUs, passing data in
        # Create a blank array for the causes
        causes = np.zeros
        for i, vpu in enumerate(self.vpus):
            start = i*self.vec_len
            end = (i+1)*self.vec_len
            r, residual = vpu.iterate(input_array[start:end])
            self.causes[i] = r
            self.residuals[start:end] = residual
        
    def get_causes(self):
        """Return output of VPUs as array."""
        return self.causes
    
    def get_residuals(self):
        """Return residual output as array."""
        return self.residuals
    
    def __repr__(self):
        """Print layer information."""
        string = f"There are {self.stage_len} units \n"
        string += f"with dimensionality {self.vec_len}x1"
        return string
            
        

In [20]:
stages = TimeStage(3, 10)
assert len(stages.vpus) == 10
assert not stages.causes.any()
assert not stages.residuals.any()
print(stages)

There are 10 units 
with dimensionality 3x1


In [78]:
data_in = np.random.randint(2, size=(stages.size, 1))
print(data_in.T)

stages.forward(data_in)

[[0 1 1 1 1 0 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 1 1 1 1 0 1 0 0]]


In [79]:
stages.causes

array([[1.16255087],
       [1.18229776],
       [0.        ],
       [0.69690351],
       [1.65516967],
       [0.        ],
       [0.55931224],
       [1.27483635],
       [0.95252383],
       [0.6175415 ]])

In [80]:
stages.residuals

array([[-0.65255676],
       [ 0.24554551],
       [ 0.40292996],
       [ 0.20690901],
       [ 0.395263  ],
       [-0.63492349],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.51432549],
       [-0.08843269],
       [-0.49190897],
       [ 0.41373736],
       [ 0.12016362],
       [-0.27348761],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [-0.30632323],
       [ 0.68716982],
       [-0.34804244],
       [ 0.11131143],
       [ 0.10060479],
       [ 1.16287605],
       [ 0.64792454],
       [ 0.4447738 ],
       [-0.68925206],
       [ 0.6186425 ],
       [-0.25279038],
       [-0.41475412]])

We need a method to "bed in" the covariance - we need to input data for a certain number of iterations.

In [82]:
"9" in stages.__repr__()

False

In [ ]:
for _ in range(0, 100):
    in_1 = np.random.randint(255, size=(2, 1))
    in_1 = in_1 / in_1.max()
    vpu.update_cov(in_1)

In [ ]:
class Sensor:
    """Object to process a 1D sensor signal."""

    def __init__(self, sensor_source):
        """Initialise sensor.

        Arg:
            sensor_source - SensorSource object that outputs a
            vector of sensor readings when iterated.
        """
        